In [1]:
import numpy as np
from random import randint
from math import floor, fabs
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys

def SecModelParameter ( VarsValue, m, n, my_rhs, my_coef, BranchedVariables):
#     print 'BranchedVariables',BranchedVariables
    unbranched=list()
    c=[]
    d=(np.zeros(((2 * m) + (4 * n), 2*n)))
    halatest=[]
    a=[]
    newrow=[]
    zer=(np.zeros(((2 * m) + (4 * n), 2*n)))
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)
    NodeNum=n-len(unbranched)
    
    
    for i in unbranched:   
        for h in range (m):
            c.append(0)
        for h in range (m):
            lin=0            
            for j in range (n):
#                 print 'i', i, 'h', h,'j',j
#                 print 'j1',j
                if j in (BranchedVariables):
#                     print 'j2',j
                    sv = VarsValue[BranchedVariables.index(j)]
#                     print 'sv', sv
                    lin =lin+ (sv * my_coef[h][j]) 
#                     print 'lin', lin
#                 print 'sv'
            lin=lin- my_rhs[h]
            c.append(lin)                
        for h in range (n):
            c.append(0)
        for h in range (n):
            c.append(0)

        for jj in range (n):
            lin2=-1
            if jj in (BranchedVariables):
                sv = VarsValue[BranchedVariables.index(jj)]
                lin2 =lin2+ sv
            c.append(lin2)
            
        for jj in range (n):
            lin2=0
            if jj in (BranchedVariables):
                lin2 =-1*VarsValue[BranchedVariables.index(jj)]
            c.append(lin2)

              
    k=NodeNum-1 

    for i in unbranched: 
        k=k+1
        for t in range(m):              
            for j in range(n):    
                d[(k-NodeNum)*(2*m+4*n)+t][j+n]=my_coef[t][j] 
                
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t][n+i]=d[(k-NodeNum)*(2*m+4*n)+t][n+i]-my_rhs[t]   
                
        for t in range(m):  
            for kk in unbranched:
                d[(k-NodeNum)*(2*m+4*n)+t+m][kk]=my_coef[t][kk] 
                

        for t in range(m):  
            for j in range(n):
                d[(k-NodeNum)*(2*m+4*n)+t+m][j+n]=-1*my_coef[t][j] 
                 
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]=d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]+my_rhs[t] 
                
        for t in range(n):              
            d[(k-NodeNum)*(2*m+4*n)+t+2*m][i]=-1 
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+t][n+t]=1 
        
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+n+t][n+t]=-1 
        
        for mm in unbranched: 
             
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+mm][mm]=1  
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]=d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]+1            
            

        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+t][n+t]=-1 

        for t in unbranched:  
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][t]=-1  
               
        
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][n+t]=1 
            
        d=np.vstack([d, zer])                          
            
    d = np.delete(d, slice(((n - NodeNum)*((2 * m) + (4 * n))),((n-NodeNum+1)*((2 * m) + (4 * n)))), axis=0)
#     print 'c',c
#     print 'unbranched',unbranched
    return c, d,unbranched
#     print c, d,unbranched

In [2]:
def SecondModel (cc,dd,NotBranched):
#     print 'cc,dd,NotBranched',cc,dd,NotBranched
    numrows = len(dd)    # 3 rows in your example
    numcols = len(dd[0])
    VarNum=numcols/2
    ConstNum=numrows/len(NotBranched)
    my_obj=[]
    my_ub=[]
    my_lb=[]
    my_colnames=[]
    my_rownames=["c1"]
    my_coef=[] 
    my_row=[]
    my_rhs=[]
    my_sense=""
    my_Nmah=list()
    
    for i in range(numrows):
        my_obj=cc
        my_ub.append(1)
        my_lb.append(0)
        my_colnames.append("y"+str(i))

    sumarray=[]     
    NotBranched.sort()    
#     sumshode=list()
    for t in range (VarNum):
        sumlist=list()
        for j in range(numrows):
#             print 't,j',t,j
            if t in NotBranched: 
                indj=NotBranched.index(t)
#                 print 'indj',indj
                indjrange=range(indj*ConstNum,(indj+1)*ConstNum)
#                 print 'indjrange',indjrange
                if j in indjrange:
#                     print 'dd[j,t]+dd[j,t+VarNum]',j,'ll',dd[j,t]+dd[j,t+VarNum]
                    sumlist.append(dd[j,t]+dd[j,t+VarNum])
                    dd[j,t+VarNum]=0
                else:
                    sumlist.append(dd[j,t])
            else:
                sumlist.append(dd[j,t])
        sumarray.append(sumlist)
#     print 'sumarray',sumarray
#     print 'newDD',dd
  
#     print 'notbranched',NotBranched
    lenNotbranched=len(NotBranched)
   
    
    for t in range(lenNotbranched):  
        for i in range (VarNum):
            sumlist=list()
            if t>0:
                for p in range(t):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                        
            for j in range(ConstNum):
                sumlist.append(dd[t*ConstNum+j,i+VarNum])
            if i in NotBranched and i > NotBranched[t]:
                indx=NotBranched.index(i)
                indy=NotBranched.index(NotBranched[t])
                
                for x in range(indx-indy-1):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                for j in range(ConstNum):
                    sumlist.append(dd[(indx)*ConstNum+j,NotBranched[t]+VarNum])
                    dd[(indx)*ConstNum+j,NotBranched[t]+VarNum]=0
#                 sumarray.append([t+indx,indy])
                if t<lenNotbranched-2 :
                    for p in range(lenNotbranched-indx-1):
                        for j in range (ConstNum):
                            sumlist.append(0)
            else:
                if t<lenNotbranched-1 :
#                     print 'lenNotbranched',lenNotbranched
                    for p in range(lenNotbranched-t-1):
                            for j in range (ConstNum):
                                sumlist.append(0) 
#             print 't,i',t,',',i
#             print 'sumlist p2',sumlist
#             print 'lensumlist p2',len(sumlist)
            sumarray.append(sumlist)
#     print 'part2' ,sumarray  
    
    rows=[]
    for j in range(numrows):
        my_row.append(1)
    rows.append([range(numrows),my_row])  
    def populatebyrow(SecProb):
        SecProb.objective.set_sense(SecProb.objective.sense.maximize)
        SecProb.variables.add(obj = my_obj,lb=my_lb, names = my_colnames)
        
        for i in range (len(sumarray)):
            if all(k == 0 for k in sumarray[i])!=True:
#                 print 'i',i
                sumline=[]
                sumline.append([range(len(sumarray[i])),sumarray[i]])
#                 print 'sumline',sumline
                SecProb.linear_constraints.add(lin_expr = sumline, senses =  "E",
                                rhs = [0], names = ["u"+str(i)])
        SecProb.linear_constraints.add(lin_expr = rows, senses = "L",
                                             rhs = [1], names = ["t0"])
    
    my_SecProb = CPX.Cplex()
    handle = populatebyrow(my_SecProb)
    my_SecProb.solve()
    my_SecProb.write("lpex2.lp")
    SecVariables    = my_SecProb.solution.get_values()
    SecObjective=my_SecProb.solution.get_objective_value()
#     print 'SecVariables,SecObjective,my_colnames',SecVariables, SecObjective,my_colnames
    return SecVariables, SecObjective,my_colnames

In [3]:
def GeneratintCutParameters ( m, n, my_rhs, my_coef, BranchedVariables,SecVariables): 
    BranchedLen=len(BranchedVariables)
    unbranched=list()
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)  
    LeUnbranched=len(unbranched)                   
#     NodeNum=n-len(unbranched)
#     k=NodeNum-1 
    lhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),BranchedLen)))
    rhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),1)))

    for i in range (LeUnbranched):   
        for h in range (m):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+m+h][BranchedVariables.index(j)]=my_coef[h][j]*SecVariables[i*(2*m+4*n)+m+h]  
                rhs[i*(2*m+4*n)+m+h]=my_rhs[h]*SecVariables[i*(2*m+4*n)+m+h]  
            
        for h in range (n):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+2*m+2*n+j][BranchedVariables.index(j)]=SecVariables[i*(2*m+4*n)+2*m+2*n+j]  
                rhs[i*(2*m+4*n)+2*m+2*n+h]=SecVariables[i*(2*m+4*n)+2*m+2*n+h]  
            
#         for h in range (n):
        for j in (BranchedVariables):
            lhs[i*(2*m+4*n)+2*m+3*n+j][BranchedVariables.index(j)]=-1*SecVariables[i*(2*m+4*n)+2*m+3*n+j]  

    #     print 'lefttt',lhs,'rightttt', rhs
    LenLeft=len(lhs)
    LenRight=len(rhs)
    LenSec=len(SecVariables)
#     print 'LenLeft',LenLeft
#     print'LenRight',LenRight
#     print 'LenSec',LenSec
#     indicesLhs=[]
#     for j in range (8):
#         indicesLhs.append([i for i in range(LenLeft) if lhs[i][j]!=0]) 
#     print indicesLhs
#     indicesRhs = [(i,rhs[i]) for i in range(LenRight) if rhs[i]>0]
#     print 'indicesRhs',indicesRhs
#     indiceSecVariables=[(i , SecVariables[i]) for i in range(LenSec) if SecVariables[i]!=0]
#     ValueSecVariables=[SecVariables[i] for i in range(LenSec) if SecVariables[i]!=0]
#     print 'positiveLeftHand',indicesLhs 
#     print 'indiceSecVariables',indiceSecVariables
#     print 'ValueSecVariables', ValueSecVariables
    FinalLhs=np.sum(lhs, axis=0)
    FinalRhs=np.sum(rhs, axis=0)
    print 'FinalLhs',FinalLhs, 'FinalRhs',FinalRhs
    return FinalLhs,FinalRhs



In [4]:
class MyCut(CPX_CB.UserCutCallback):

    def __call__(self):
         
        self.times_called += 1
        print  'self.times_called', self.times_called
        branched=list()
        values=list()
        branchedNames=list()
        global BranchedMatrix
        senses= "L"
#         print 'self.get_lower_bounds(s)',self.get_lower_bounds()
#         print 'self.get_upper_bounds(s)',self.get_upper_bounds()
#         print 'objective value', self.get_objective_value()
#         print 'values', self.get_values()
        print 'get_num_nodes', self.get_num_nodes()

        lb=self.get_lower_bounds()
        ub=self.get_upper_bounds()               
        for j in range(len(lb)):
            if lb[j]==ub[j]:
                branched.append(j)
                values.append(lb[j])
                branchedNames.append(VarsName[j])
                
        print 'values',values
        print 'branchedNames',branchedNames
        print 'branched',branched

        if len(values)>0:  
            BranchedMatrix.append([branched,values])
#             BranchedValue.append(values)
#             print 'BranchedMatrix',BranchedMatrix

            if [BranchedMatrix[-1]]!=[BranchedMatrix[-2]] :
                SecModelParameterOutput=SecModelParameter(values, self.m, self.n, self.my_rhs, self.my_coef,branched)
                SecondModelOutput=SecondModel (SecModelParameterOutput[0],SecModelParameterOutput[1],SecModelParameterOutput[2])
                print 'SecondModelOutput[1]',SecondModelOutput[1]
                print 'SecondModelOutput[0]',SecondModelOutput[0]
                if SecondModelOutput[1]>0.000001:
    #                 print 'bayad cut bezane'
                    output=GeneratintCutParameters (self.m, self.n, self.my_rhs, self.my_coef,branched, SecondModelOutput[0])
                    print 'output',output[0]
                    print 'output[1]',output[1]
                    lhs=[CPX.SparsePair(ind =branchedNames, val = output[0])]
                    print 'lhs',lhs[0]
                    self.add(cut=lhs[0],rhs=output[1][0],sense=senses)
                    print 'cut added'



In [5]:
def admipex3(c):
    
    c.parameters.preprocessing.presolve = 0

    c.parameters.preprocessing.reduce = 0

    c.parameters.preprocessing.linear.set(0)

    c.parameters.preprocessing.numpass = 0

    c.parameters.mip.strategy.presolvenode = -1

    c.parameters.preprocessing.linear = 0
    
def CutCplex(c):
    c.parameters.mip.cuts.disjunctive.set(-1)
    c.parameters.mip.cuts.bqp.set(-1)
    c.parameters.mip.cuts.cliques.set(-1)
    c.parameters.mip.cuts.covers.set(-1)
    c.parameters.mip.cuts.flowcovers.set(-1)
    c.parameters.mip.cuts.gomory.set(-1)
    c.parameters.mip.cuts.gubcovers.set(-1)
    c.parameters.mip.cuts.implied.set(-1)
    c.parameters.mip.cuts.liftproj.set(-1)
    c.parameters.mip.cuts.localimplied.set(-1)
    c.parameters.mip.cuts.mcfcut.set(-1)
    c.parameters.mip.cuts.mircut.set(-1)
    c.parameters.mip.cuts.zerohalfcut.set(-1)
    c.parameters.mip.cuts.pathcut.set(-1)
    
c = CPX.Cplex('NewExample.lp')
m=c.linear_constraints.get_num()
n=c.variables.get_num()
my_rhs=[]
my_coef=[]
for i in range(m):
    my_row=[]
    for j in range(n):
        my_row.append(c.linear_constraints.get_coefficients(i,j))
    my_coef.append(my_row)

for i in range (m):
    my_rhs.append(c.linear_constraints.get_rhs(i))

admipex3(c)
VarsName=c.variables.get_names()
c.set_log_stream(sys.stdout)
c.set_results_stream(sys.stdout)

cutinst=c.register_callback(MyCut)
cutinst.times_called = 0
cutinst.m=m
cutinst.n=n
cutinst.my_rhs=my_rhs
cutinst.my_coef=my_coef
cutinst.VarsName=VarsName
BranchedMatrix=[[[-1], [-0.0]]]
c.parameters.mip.interval.set(1)
c.parameters.preprocessing.linear=0
c.parameters.mip.strategy.search.set(c.parameters.mip.strategy.search.values.traditional)
c.solve()

c.write("lpex1.lp")
solution = c.solution

# the following line prints the corresponding string
print solution.status[solution.get_status()]
print "Objective value = " , solution.get_objective_value()
print
x = solution.get_values(0, c.variables.get_num()-1)
for j in range(c.variables.get_num()):
    if fabs(x[j]) > 1.0e-10:
        print "Column %d: Value = %17.10g" % (j, x[j])

print "Branch callback was called ", cutinst.times_called, "times"


CPXPARAM_Read_DataCheck                          1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Interval                            1
CPXPARAM_MIP_Strategy_Search                     1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 25 rows, 25 columns, and 621 nonzeros.
Reduced MIP has 25 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.17 ticks)
Probing time = 0.00 sec. (0.03 ticks)
Tried aggregator 1 time.
Reduced MIP has 25 rows, 25 columns, and 621 nonzeros.
Reduced MIP has 25 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.24 ticks)
Probing time = 0.00 sec. (0.03 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.14 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  

      1     3      827.2127    11      730.0000      827.1696      101   13.31%             x10 D      1      0      1
self.times_called 19
get_num_nodes 2
values [-0.0, 1.0]
branchedNames ['x10', 'x15']
branched [10, 15]
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 1058 columns.
Reduced LP has 323 rows, 2392 columns, and 45874 nonzeros.
Presolve time = 0.02 sec. (4.47 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          1208.658752
Iteration:    63   Scaled dual infeas =            35.197891
Iteration:   125   Scaled dual infeas =            35.197891
Iteration:   187   Scaled dual infeas =            35.197891
Perturbation started.
Iteration:   202   Scaled dual infeas =            35.198022
Iteration:   264   Scaled dual infeas =            35.197890
Iteration:   326   Scaled dual

values [-0.0, 1.0]
branchedNames ['x10', 'x15']
branched [10, 15]
      2     4      826.9588    11      730.0000      825.4669      104   13.08%             x15 U      2      1      2
self.times_called 21
get_num_nodes 3
values [-0.0, -0.0]
branchedNames ['x10', 'x15']
branched [10, 15]
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 1058 columns.
Reduced LP has 323 rows, 2392 columns, and 45874 nonzeros.
Presolve time = 0.02 sec. (4.47 ticks)
Initializing dual steep norms . . .

Dual simplex solved model.

SecondModelOutput[1] 0.0
SecondModelOutput[0] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

      3     5      810.2720     7      730.0000      825.4669      116   13.08%             x15 D      3      1      2
self.times_called 23
get_num_nodes 4
values [-0.0, -0.0, 1.0]
branchedNames ['x5', 'x10', 'x15']
branched [5, 10, 15]
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 968 columns.
Reduced LP has 320 rows, 2332 columns, and 43437 nonzeros.
Presolve time = 0.01 sec. (4.23 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          1168.811697
Iteration:    62   Scaled dual infeas =            47.652581
Iteration:   124   Scaled dual infeas =            47.652581
Iteration:   186   Scaled dual infeas =            47.652581
Perturbation started.
Iteration:   202   Scaled dual infeas =            47.652701
Iteration:   264   Scaled dual infeas =            39.413523
Iteration:   326

      4     6      818.1828     8      730.0000      825.4669      124   13.08%              x5 D      4      2      3
self.times_called 25
get_num_nodes 5
values [-0.0, 1.0, -0.0, 1.0]
branchedNames ['x5', 'x6', 'x10', 'x15']
branched [5, 6, 10, 15]
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 882 columns.
Reduced LP has 316 rows, 2268 columns, and 41022 nonzeros.
Presolve time = 0.02 sec. (4.00 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          3695.623397
Iteration:    62   Scaled dual infeas =           292.912151
Iteration:   124   Scaled dual infeas =           186.369089
Iteration:   186   Scaled dual infeas =            19.045523
Iteration:   248   Scaled dual infeas =            19.045523
Iteration:   310   Scaled dual infeas =            15.560030
Iteration:   372   Scale

...

    200    16      750.9036     6      730.0000      756.4193     1010    3.62%              x1 N    200     54      7
Elapsed time = 115.64 sec. (35.51 ticks, tree = 0.01 MB, solutions = 3)
    201    15        cutoff            730.0000      756.4193     1012    3.62%             x22 U    201    200      8
    202    14        cutoff            730.0000      756.4193     1014    3.62%             x22 D    202    200      8
self.times_called 292
get_num_nodes 203
values [-0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x5', 'x6', 'x10', 'x12', 'x14', 'x15', 'x17', 'x18', 'x19', 'x20', 'x24']
branched [1, 5, 6, 10, 12, 14, 15, 17, 18, 19, 20, 24]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 338 columns.
Reduced LP has 248 rows, 1612 columns, and 23234 nonzeros.
Presolve time = 0.02 sec. (2.28 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dua

FinalLhs [-0.06091869 -0.00492014 -0.02635093 -0.05612581 -0.0323413   0.05761191
  0.07733416  0.02096847  0.03139325  0.07650368  0.01214527  0.09579364] FinalRhs [0.36958924]
output [-0.06091869 -0.00492014 -0.02635093 -0.05612581 -0.0323413   0.05761191
  0.07733416  0.02096847  0.03139325  0.07650368  0.01214527  0.09579364]
output[1] [0.36958924]
lhs SparsePair(ind = ['x1', 'x5', 'x6', 'x10', 'x12', 'x14', 'x15', 'x17', 'x18', 'x19', 'x20', 'x24'], val = array([-0.06091869, -0.00492014, -0.02635093, -0.05612581, -0.0323413 ,
        0.05761191,  0.07733416,  0.02096847,  0.03139325,  0.07650368,
        0.01214527,  0.09579364]))
cut added
    203    13    infeasible            730.0000      756.3724     1021    3.61%             x14 N    203     78      7
    204    12        cutoff            730.0000      755.4299     1023    3.48%              x9 D    204    143      8
    205    11        cutoff            730.0000      755.1426     1026    3.44%             x14 U    205    

values [1.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x1', 'x5', 'x9', 'x10', 'x11', 'x16', 'x17', 'x18', 'x19', 'x20', 'x23', 'x24']
branched [1, 5, 9, 10, 11, 16, 17, 18, 19, 20, 23, 24]
    206    12      747.5404     8      730.0000      754.9175     1031    3.41%             x17 N    206    106      6
self.times_called 296
get_num_nodes 207
values [1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0]
branchedNames ['x1', 'x3', 'x5', 'x9', 'x10', 'x11', 'x14', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x23', 'x24']
branched [1, 3, 5, 9, 10, 11, 14, 16, 17, 18, 19, 20, 21, 23, 24]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 200 columns.
Reduced LP has 206 rows, 1300 columns, and 17243 nonzeros.
Presolve time = 0.00 sec. (1.70 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =           850.000000
Iteration

    207    13      746.2686     8      730.0000      754.9175     1033    3.41%             x14 D    207    206      7
self.times_called 298
get_num_nodes 208
values [-0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0]
branchedNames ['x0', 'x1', 'x3', 'x5', 'x9', 'x10', 'x11', 'x14', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x23', 'x24']
branched [0, 1, 3, 5, 9, 10, 11, 14, 16, 17, 18, 19, 20, 21, 23, 24]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 162 columns.
Reduced LP has 190 rows, 1188 columns, and 15330 nonzeros.
Presolve time = 0.01 sec. (1.51 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =           579.698252
Iteration:    62   Scaled dual infeas =             9.634673
Iteration:   124   Scaled dual infeas =             6.770963
Iteration:   186   Scaled dual infeas =             0.097474
Iteration:   244   Dual objective

    208    12      739.4792     5      730.0000      754.9175     1039    3.41%              x0 D    208    207      8
    209    11        cutoff            730.0000      754.9175     1041    3.41%              x0 U    209    207      8
    210    10        cutoff            730.0000      748.1263     1043    2.48%             x22 D    210    152      6
Elapsed time = 117.88 sec. (36.45 ticks, tree = 0.01 MB, solutions = 3)
    211     9    infeasible            730.0000      747.2005     1043    2.36%              x1 D    211     54      7
    212     8        cutoff            730.0000      746.9335     1048    2.32%             x24 D    212    159     10
    213     7        cutoff            730.0000      745.0912     1052    2.07%             x14 U    213    206      7
    214     6        cutoff            730.0000      744.0477     1053    1.92%             x12 U    214    122      7
    215     5        cutoff            730.0000      742.7086     1055    1.74%             x12